In [1]:
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd

In [2]:
# establecemos la conexión con el terminal MetaTrader 5
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
train_from = datetime(2017, 12, 31)
train_to = datetime(2019, 1, 1)
# solicitamos los ticks de USDJPY en el intervalo 01.01.2018 - 01.01.2019
train_rates = mt5.copy_rates_range("USDJPY", mt5.TIMEFRAME_MN1, train_from, train_to)

In [4]:
# creamos un DataFrame de los datos obtenidos
train_frame = pd.DataFrame(train_rates)
# convertimos la hora en segundos al formato datetime
train_frame['time']=pd.to_datetime(train_frame['time'], unit='s')

In [5]:
train_frame

,time,open,high,low,close,tick_volume,spread,real_volume
0,2018-01-01,112.503,113.381,108.277,109.181,11466322,6,2293983495000
1,2018-02-01,109.174,110.477,105.541,106.668,11719224,6,2343961380000
2,2018-03-01,106.668,107.284,104.625,106.275,13202801,6,2640566400000
3,2018-04-01,106.164,109.528,105.653,109.336,14260906,6,2849060050000
4,2018-05-01,109.336,111.390,108.107,108.809,13797751,6,2759550200000
5,2018-06-01,108.809,110.940,108.715,110.756,12302362,6,2460472400000
6,2018-07-01,110.617,113.167,110.272,111.809,12530746,6,2506149200000
7,2018-08-01,111.799,112.146,109.769,111.103,12632511,6,2526502200000
8,2018-09-01,111.088,113.706,110.374,113.698,10085120,6,2017024000000
9,2018-10-01,113.691,114.543,111.371,112.927,14977781,6,2995471800000


In [8]:
test_from = datetime(2019, 1, 1)
test_to = datetime(2020, 2, 1)
# solicitamos los ticks de USDJPY en el intervalo 01.02.2019 - 01.02.2020
test_rates = mt5.copy_rates_range("USDJPY", mt5.TIMEFRAME_MN1, test_from, test_to)

In [9]:
# creamos un DataFrame de los datos obtenidos
test_frame = pd.DataFrame(test_rates)
# convertimos la hora en segundos al formato datetime
test_frame['time']=pd.to_datetime(test_frame['time'], unit='s')
test_frame

,time,open,high,low,close,tick_volume,spread,real_volume
0,2019-02-01,108.828,111.485,108.718,111.372,14374382,13,2156157300000
1,2019-03-01,111.369,112.129,109.704,110.849,18863659,12,2829548850000
2,2019-04-01,111.014,112.395,110.803,111.407,17316609,12,2597491350000
3,2019-05-01,111.407,111.689,108.271,108.279,21498736,13,3224810400000
4,2019-06-01,108.190,108.794,106.771,107.821,18295554,13,2744333100000
5,2019-07-01,108.142,108.995,107.203,108.754,16742425,12,2511363750000
6,2019-08-01,108.754,109.311,104.441,106.278,27675245,12,4151286750000
7,2019-09-01,106.061,108.470,105.733,108.067,16501681,12,2475252150000
8,2019-10-01,108.067,109.281,106.476,108.013,31466438,13,4719965700000
9,2019-11-01,108.013,109.662,107.880,109.509,18428138,13,2764220700000


In [ ]:
hoy = pd.Timestamp(datetime.datetime.today())
for t in range(1, len(ma_df)):
    # For Moving Average
    ## Compras
    if (ma_df.loc[t-1, 'moving_average_short'] < ma_df.loc[t-1, 'moving_average_long'] and
        ma_df.loc[t, 'moving_average_short'] > ma_df.loc[t, 'moving_average_long'] and
        ma_df.loc[t, 'close'] * num_acciones <= Capital):
        final_ma.loc[t, 'Operation'] = 'Comprar'
        final_ma.loc[t, 'exposure'] = ma_df.loc[t, 'close'] * num_acciones
        # Cálculo stoploss & takeprofit
        final_ma.loc[t, 'Stoploss'] = max(ma_df.loc[t, 'close'] - ((Capital * 0.1) / num_acciones),
                                          ma_df.loc[t, 'close'] * (1 - pp_ma))
        final_ma.loc[t, 'Takeprofit'] = ma_df.loc[t, 'close'] * (1 + pg_ma)
        # Actualización Capital
        Capital = Capital - (ma_df.loc[t, 'close'] * num_acciones)
                
        # Cerrando posición
        ## Precio Stoploss
        close_sl = ma_df[(ma_df['time'] >= final_ma.loc[t, 'time']) &
                         (ma_df['close'] <= final_ma.loc[t, 'Stoploss'])].close.to_list()
        close_sl.append([0])
        final_ma.loc[t, 'SL'] = close_sl[0]
        
        ## Precio Takeprofit
        close_tp = ma_df[(ma_df['time'] >= final_ma.loc[t, 'time']) &
                         (ma_df['close'] >= final_ma.loc[t, 'Takeprofit'])].close.to_list()
        close_tp.append([0])
        final_ma.loc[t, 'TP'] = close_tp[0]
        
        ## Time Stoploss
        time_sl = ma_df[(ma_df['time'] >= final_ma.loc[t, 'time']) &
                         (ma_df['close'] <= final_ma.loc[t, 'Stoploss'])].time.to_list()
        time_sl.append([hoy])
        final_ma.loc[t, 'Date_SL'] = time_sl[0]

        
        ## Time Takeprofit
        time_tp = ma_df[(ma_df['time'] >= final_ma.loc[t, 'time']) &
                         (ma_df['close'] >= final_ma.loc[t, 'Takeprofit'])].time.to_list()
        time_tp.append([hoy])        
        final_ma.loc[t, 'Date_TP'] = time_tp[0]        
        
        # Operación cerrada
        final_ma.loc[t, 'CloseOp'] = np.where(final_ma.loc[t, 'Date_SL'] > final_ma.loc[t, 'Date_TP'],
                                     final_ma.loc[t, 'TP'],
                                     final_ma.loc[t, 'SL'])
        
        # Profit
        final_ma.loc[t, 'Profit'] = (final_ma.loc[t, 'CloseOp'] - final_ma.loc[t, 'close']) * num_acciones if \
                                     final_ma.loc[t, 'CloseOp'] != 0 else 0
        
        # Actualización Capital después de cerrar posición
        Capital = Capital + (final_ma.loc[t, 'CloseOp'] * num_acciones)
        
        
    ## Ventas
    elif (ma_df.loc[t-1, 'moving_average_short'] > ma_df.loc[t-1, 'moving_average_long'] and
          ma_df.loc[t, 'moving_average_short'] < ma_df.loc[t, 'moving_average_long'] and
          ma_df.loc[t, 'close'] * num_acciones <= Capital):
        final_ma.loc[t, 'Operation'] = 'Vender'
        final_ma.loc[t, 'exposure'] = ma_df.loc[t, 'close'] * num_acciones
        # Cálculo stoploss & takeprofit
        final_ma.loc[t, 'Stoploss'] = min(ma_df.loc[t, 'close'] + ((Capital * 0.1) / num_acciones),
                                          ma_df.loc[t, 'close'] * (1 + pp_ma))
        final_ma.loc[t, 'Takeprofit'] = ma_df.loc[t, 'close'] * (1 - pg_ma)
        # Actualización Capital
        Capital = Capital - ma_df.loc[t, 'close'] * num_acciones
        # Cerrando posición
        ## Precio Stoploss
        close_sl = ma_df[(ma_df['time'] >= final_ma.loc[t, 'time']) &
                         (ma_df['close'] >= final_ma.loc[t, 'Stoploss'])].close.to_list()
        close_sl.append([0])
        final_ma.loc[t, 'SL'] = close_sl[0]
        ## Precio Takeprofit
        close_tp = ma_df[(ma_df['time'] >= final_ma.loc[t, 'time']) &
                         (ma_df['close'] <= final_ma.loc[t, 'Takeprofit'])].close.to_list()
        close_tp.append([0])
        final_ma.loc[t, 'TP'] = close_tp[0]
        ## Time Stoploss
        time_sl = ma_df[(ma_df['time'] >= final_ma.loc[t, 'time']) &
                         (ma_df['close'] >= final_ma.loc[t, 'Stoploss'])].time.to_list()
        time_sl.append([hoy])
        final_ma.loc[t, 'Date_SL'] = time_sl[0]
        ## Time Takeprofit
        time_tp = ma_df[(ma_df['time'] >= final_ma.loc[t, 'time']) &
                         (ma_df['close'] <= final_ma.loc[t, 'Takeprofit'])].time.to_list()
        time_tp.append([hoy])        
        final_ma.loc[t, 'Date_TP'] = time_tp[0]
        # Operación cerrada
        final_ma.loc[t, 'CloseOp'] = np.where(final_ma.loc[t, 'Date_SL'] > final_ma.loc[t, 'Date_TP'],
                                     final_ma.loc[t, 'TP'],
                                     final_ma.loc[t, 'SL'])
        # Profit
        final_ma.loc[t, 'Profit'] = (final_ma.loc[t, 'close'] - final_ma.loc[t, 'CloseOp']) * num_acciones if \
                                     final_ma.loc[t, 'CloseOp'] != 0 else 0
        # Actualización Capital después de cerrar posición
        Capital = Capital + (final_ma.loc[t, 'CloseOp'] * num_acciones)
    
    # For Boolinger Bands
    ## Compras
    if (bb_df.loc[t-1, 'close'] > bb_df.loc[t-1, 'boolinger_lower'] and
        bb_df.loc[t, 'close'] < bb_df.loc[t, 'boolinger_lower'] and
        bb_df.loc[t, 'close'] * num_acciones <= Capital and
        ((final_bb.loc[t, 'time'] > final_bb.loc[t-1, 'time'] + datetime.timedelta(hours=1)) and 
        (final_bb.loc[t-1, 'Operation'] != "Comprar"))):
        final_bb.loc[t, 'Operation'] = 'Comprar'
        final_bb.loc[t, 'exposure'] = bb_df.loc[t, 'close'] * num_acciones
        # Cálculo stoploss & takeprofit
        final_bb.loc[t, 'Stoploss'] = max(bb_df.loc[t, 'close'] - ((Capital * 0.1) / num_acciones),
                                          bb_df.loc[t, 'close'] - (bb_df.loc[t, 'moving_average'] - bb_df.loc[t, 'close']))
        final_bb.loc[t, 'Takeprofit'] = bb_df.loc[t, 'moving_average']           
        # Actualización Capital
        Capital = Capital - bb_df.loc[t, 'close'] * num_acciones
        # Cerrando posición
        ## Precio Stoploss
        close_sl = bb_df[(bb_df['time'] >= final_bb.loc[t, 'time']) &
                         (bb_df['close'] <= final_bb.loc[t, 'Stoploss'])].close.to_list()
        close_sl.append([0])
        final_bb.loc[t, 'SL'] = close_sl[0]
        ## Precio Takeprofit
        close_tp = bb_df[(bb_df['time'] >= final_bb.loc[t, 'time']) &
                         (bb_df['close'] >= final_bb.loc[t, 'Takeprofit'])].close.to_list()
        close_tp.append([0])
        final_bb.loc[t, 'TP'] = close_tp[0]
        ## Time Stoploss
        time_sl = bb_df[(bb_df['time'] >= final_bb.loc[t, 'time']) &
                         (bb_df['close'] <= final_bb.loc[t, 'Stoploss'])].time.to_list()
        time_sl.append([hoy])
        final_bb.loc[t, 'Date_SL'] = time_sl[0]
        ## Time Takeprofit
        time_tp = bb_df[(bb_df['time'] >= final_bb.loc[t, 'time']) &
                         (bb_df['close'] >= final_bb.loc[t, 'Takeprofit'])].time.to_list()
        time_tp.append([hoy])        
        final_bb.loc[t, 'Date_TP'] = time_tp[0]
        # Operación cerrada
        final_bb.loc[t, 'CloseOp'] = np.where(final_bb.loc[t, 'Date_SL'] > final_bb.loc[t, 'Date_TP'],
                                     final_bb.loc[t, 'TP'],
                                     final_bb.loc[t, 'SL'])
        # Profit
        final_bb.loc[t, 'Profit'] = (final_bb.loc[t, 'CloseOp'] - final_bb.loc[t, 'close']) * num_acciones if \
                                     final_bb.loc[t, 'CloseOp'] != 0 else 0
        # Actualización Capital después de cerrar posición
        Capital = Capital + (final_bb.loc[t, 'CloseOp'] * num_acciones)
    ## Ventas
    elif (bb_df.loc[t-1, 'close'] < bb_df.loc[t-1, 'boolinger_upper'] and
          bb_df.loc[t, 'close'] > bb_df.loc[t, 'boolinger_upper'] and
          bb_df.loc[t, 'close'] * num_acciones <= Capital and
          ((final_bb.loc[t, 'time'] > final_bb.loc[t-1, 'time'] + datetime.timedelta(hours=1)) and 
          (final_bb.loc[t-1, 'Operation'] != "Vender"))):
        final_bb.loc[t, 'Operation'] = 'Vender'
        final_bb.loc[t, 'exposure'] = bb_df.loc[t, 'close'] * num_acciones
        # Cálculo stoploss & takeprofit
        final_bb.loc[t, 'Stoploss'] = min(bb_df.loc[t, 'close'] + ((Capital * 0.1) / num_acciones),
                                          bb_df.loc[t, 'close'] + (bb_df.loc[t, 'close'] - bb_df.loc[t, 'moving_average']))
        final_bb.loc[t, 'Takeprofit'] = bb_df.loc[t, 'moving_average']  
        # Actualización Capital
        Capital = Capital - bb_df.loc[t, 'close'] * num_acciones
        # Cerrando posición
        ## Precio Stoploss
        close_sl = bb_df[(bb_df['time'] >= final_bb.loc[t, 'time']) &
                         (bb_df['close'] >= final_bb.loc[t, 'Stoploss'])].close.to_list()
        close_sl.append([0])
        final_bb.loc[t, 'SL'] = close_sl[0]
        ## Precio Takeprofit
        close_tp = bb_df[(bb_df['time'] >= final_bb.loc[t, 'time']) &
                         (bb_df['close'] <= final_bb.loc[t, 'Takeprofit'])].close.to_list()
        close_tp.append([0])
        final_bb.loc[t, 'TP'] = close_tp[0]
        ## Time Stoploss
        time_sl = bb_df[(bb_df['time'] >= final_bb.loc[t, 'time']) &
                         (bb_df['close'] >= final_bb.loc[t, 'Stoploss'])].time.to_list()
        time_sl.append([hoy])
        final_bb.loc[t, 'Date_SL'] = time_sl[0]
        ## Time Takeprofit
        time_tp = bb_df[(bb_df['time'] >= final_bb.loc[t, 'time']) &
                         (bb_df['close'] <= final_bb.loc[t, 'Takeprofit'])].time.to_list()
        time_tp.append([hoy])        
        final_bb.loc[t, 'Date_TP'] = time_tp[0]
        # Operación cerrada
        final_bb.loc[t, 'CloseOp'] = np.where(final_bb.loc[t, 'Date_SL'] > final_bb.loc[t, 'Date_TP'],
                                     final_bb.loc[t, 'TP'],
                                     final_bb.loc[t, 'SL'])
        # Profit
        final_bb.loc[t, 'Profit'] = (final_bb.loc[t, 'close'] - final_bb.loc[t, 'CloseOp']) * num_acciones if \
                                     final_bb.loc[t, 'CloseOp'] != 0 else 0
        # Actualización Capital después de cerrar posición
        Capital = Capital + (final_bb.loc[t, 'CloseOp'] * num_acciones)